### A Demonstration of the RSA Cryptosystem

The following sequence of function calls to the `rsa` module illustrates a session in which Alice signs and encrypts a private message, and sends it to Bob over an insecure channel. Meanwhile, Eve, a passive adversary, intercepts all the information exchanged between Alice and Bob on the insecure channel. It is assumed that Alice, Bob and Eve all possess their own copy of the `rsa` module; but that only Alice and Bob possess the signing and encryption keys produced by the module to achieve confidentiality, integrity and authentication of the private message.

Values surrounded in square brackets [] are public (i.e., they can be transmitted over the insecure channel with no compromise to the security of the system, even if they are observed by Eve), and those that are not in square brackets are private (i.e., they must *not* be transmitted over the insecure channel).

The sequence starts with Alice generating her RSA key parameters using the `rsa` module's `generate_rsa_key` function, passing it a modulus size of `2048` bits. The function returns the quintuple `pA`, `qA`, `[nA]`, `d3A` and `d5A` (the suffix *A* on all the parameters identify them as belonging to Alice). Alice transmits her public modulus `[nA]` to Bob.

This implementation defines the constants `3` and `5` as the signature-verification and encryption keys, respectively, in order to minimize computational overhead. The size of these values in no way compromises the security of system; rather, that security depends solely on the size of the public modulus (the bigger, the more secure). These constants, together with `[nA]`, comprise Alice's public key, and may be observed by Eve with no loss of security.

Alice must, however, keep the values `pA` and `qA`&mdash;the 1024-bit factors of `[nA]`&mdash;secret. She must also keep `d3A` and `d5A`&mdash;her signing and decryption keys, which are the inverses of the signature-verification and encryption constants `3` and `5`&mdash;secret.

In [1]:
import sys
sys.path.append("../code/src")
import rsa

In [2]:
pA, qA, nA, d3A, d5A = rsa.generate_rsa_key(2048)

To get an idea of the size and proportions of these parameters, let's print some of them out in decimal form. Note that `[nA]`, which is the product of `pA` and `qA`, is twice the length of them individually; `pA`'s and `qA`'s ~310 decimal digits to `[nA]`'s ~620.

In [3]:
pA

162279906006798789263060886514357704162963141325666093978677436356353889844108896478379102980128194833078363078095599167689259952005251522853327420124931673742902118948474473980341728004022932234358862951149780434793037949844005006720576308633813946384553240898405121714796362630407937444330192276710444583639

In [4]:
qA

159266071851865186234631546871015293106885588826625296096417164204306628551602190129011150252278440628590591272587456198818544581874971411675781927123665060301957781866739235780813793863447117540855213274315728530296653759542944373444777775347134459668037444383140847902730442931982586039344145971848297997983

In [5]:
nA

25845683170192744806104846081528967982083567789342768352782394040861444966760080836393528425611346338322997234356549321687850882813964506171277977157437099293989122356579081896295111898298821403817343866249907560128831431723275588409394600962597670495367334078450723561579947917379978979058781416811561482737455812395296917408675885833747411558143695733939309671042063320699004756525480376434049247325784103325842077791228702902615140178922256875063341743588218253084227135852653920122791231571898340138880554288353258426478474856818069970455438766683682871397835349852991578437455057578669475734053265755829696800137

Recall that Alice requested a modulus of 2048 bits in length in her call to `generate_rsa_key`. Let's print the bit length of `[nA]` and see what we get.

In [6]:
int(nA).bit_length()

2048

What we see is that a string of 2048 bits equates to a decimal number of some 620 digits in length. We should see the same pattern if we print the bit lengths of `[nA]`'s prime factors, `pA` and `qA`; that is, 1024 bits represents a decimal number of about 310 digits.

In [7]:
int(pA).bit_length()

1024

In [8]:
int(qA).bit_length()

1024

Now let's make sure `[nA]` is indeed the product of `pA` and `qA`.

In [9]:
assert qA * pA == nA

Now it's Bob's turn to generate his RSA keys. To differentiate his keys from Alice's, we append each with the captial letter *B*.

Because Bob requested a modulus size of 2048, based on our inspection of Alice's keys above we can conclude that Bob's keys are the same size as Alice's, so we won't do the same inspection here.

In [10]:
pB, qB, nB, d3B, d5B = rsa.generate_rsa_key(2048)

Now that Alice and Bob have generated their keys, and shared with each other their public moduli `[nA]` and `[nB]`, they can proceed with a secure message exchange.

Let's say Alice wants to send Jenny's number (`8675309`) securely to Bob. First, Alice stores Jenny's number in the variable `mA`.

In [11]:
mA = "8675309"

Then Alice calls the `rsa` module's `sign` fuction to sign the message, and stores the signature in the variable `[oA]`. To sign the message, Alice needs to supply her public modulus `[nA]`, her secret signing key `d3A`, the two factors of `[nA]`, `pA` and `qA`, and the message `mA` to the rsa module's `sign` function.

In [12]:
oA = rsa.sign(d3A, pA, qA, mA)

Then Alice calls the `rsa` module's `rsa_encrypt_random_key` function to compute a session key that will be used to encrypt the private message using a symmetric cipher (e.g., AES). This function returns the pair `KA` and `[cA]`, which are, respectively, the encryption key itself, and the ciphertext of a value Bob will use to replicate the same encryption key on his side of the channel. Alice must keep the session key `KA` private, but she transmits the ciphertext `[cA]` to Bob.

In [13]:
KA, cA = rsa.encrypt_random_key(nB, 5)

Then Alice encrypts the private message using a symmetric cipher.

In a real application, Alice and Bob would use an industrial-strength cipher (e.g., AES, 3DES) for message encryption. To keep things simple, however, we will use a bitwise exclusive-or (XOR) of the message with the symmetric key `KA` to produce a ciphertext `[mAC]` of the message `mA`.

In [14]:
mAC = int.from_bytes(KA, byteorder="little") ^ int(mA)

Alice transmits the message signature `[oA]`, the message ciphertext `[mAC]` and the ciphertext of the symmetric-key material `[cA]` to Bob.

In order to decrpyt the private message, Bob must reproduce the symmetric key Alice used to encrypt it. He does this by calling the `rsa` module's `decrypt_random_key` function, passing it his private decryption key `d5B`, Alice's symmetric-key ciphertext `[cA]`, and the two prime factors of his public modulus `pB` and `qB` (using `pB` and `qB`, instead of its equivalent `[nB]`, provides for faster decryption by means of the Chinese Remainder Theorem).

In [15]:
KB = rsa.decrypt_random_key(d5B, cA, pB, qB)

Let's make sure the key Bob decrypted above is the same as the key Alice encrypted.

In [16]:
assert KB == KA

Now, Bob should be able to use the symmetric key `KB` to decrypt the message.

In [17]:
mB = int.from_bytes(KB, byteorder="little") ^ int(mAC)
assert str(mB) == mA

If the assertion above holds, we know that Bob has successfully decrypted the private message encrypted, and sent to him, by Alice; Jenny's number. Let's print it here, just for a sanity check.

In [18]:
str(mB)

'8675309'

All that is left is for Bob to verify Alice's signature, and thereby prove that the message has not been altered and, further, that it originated from Alice (or, at least, that it was signed with Alice's private key).

He does this by calling the `rsa` module's `verify` fuction, passing it Alice's public modulus `[nA]`, the constant signature-verification key `[3]`, the decrypted message `mB` (Jenny's number) and the sigature sent to him by Alice `[oA]`.

In [19]:
verified = rsa.verify(nA, 3, mB, oA)
assert verified == True
